In [210]:
!pip install pyspark

from pyspark.sql import SparkSession


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [211]:
spark = SparkSession.builder.master("local[*]").getOrCreate() #Seteaza numarul de core-uri de utilizat (* = toate)

csv_path = "/content/Students/StudentsPerformance/" #Locatia datelor noastre

df = spark.read.csv(csv_path, header=True, inferSchema=True) #Incarcarea datelor din CSV intr-un dataframe Spark

# New section

In [212]:
table_name = "my_table" #definirea unei variabile table_name
df.createOrReplaceTempView(table_name) #crearea unui view temporar
query = "SELECT * FROM my_table" #definirea unei interogari SQL
result = spark.sql(query) #rularea interogarii
result.show() #afisarea rezultatelor

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [213]:
df.printSchema() #afisarea schemei datelor

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)



In [214]:
df.describe().show()


+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|parental level of education|       lunch|test preparation course|        math score|     reading score|    writing score|
+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|  count|  1000|          1000|                       1000|        1000|                   1000|              1000|              1000|             1000|
|   mean|  null|          null|                       null|        null|                   null|            66.089|            69.169|           68.054|
| stddev|  null|          null|                       null|        null|                   null|15.163080096009454|14.600191937252223|15.19565701086966|
|    min|female|       group A|         associate's degree|free/reduced|          

In [215]:
df = df.withColumnRenamed('parental level of education', 'parentEducation')
df = df.withColumnRenamed('test preparation course', 'preparationCourse')
df = df.withColumnRenamed('math score', 'mathScore')
df = df.withColumnRenamed('reading score', 'readingScore')
df = df.withColumnRenamed('writing score', 'writingScore')
df.describe().show()
df.write.format('csv').mode('overwrite').option('header', 'true').save('/content/Students/StudentsPerformanceFormatted')


+-------+------+--------------+------------------+------------+-----------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|         mathScore|      readingScore|     writingScore|
+-------+------+--------------+------------------+------------+-----------------+------------------+------------------+-----------------+
|  count|  1000|          1000|              1000|        1000|             1000|              1000|              1000|             1000|
|   mean|  null|          null|              null|        null|             null|            66.089|            69.169|           68.054|
| stddev|  null|          null|              null|        null|             null|15.163080096009454|14.600191937252223|15.19565701086966|
|    min|female|       group A|associate's degree|free/reduced|        completed|                 0|                17|               10|
|    max|  male|       group E|  s

In [216]:
csv_path = "/content/Students/StudentsPerformanceFormatted" #salvam locatia csv-ului formatat intr-o noua variabila
df = spark.read.csv(csv_path, header=True, inferSchema=True) #incarcam noile date

from pyspark.sql.functions import col #importam lucrul cu coloane
df = df.withColumn('finalScore', (col('mathScore') + col('readingScore') + col('writingScore'))/3) #mai adaugam o coloana care determina media generala
df = df.withColumn('finalScore', format_number(col('finalScore'), 2))#afisam doar 2 decimale in aceasta noua coloana
df = df.withColumn('admited', when(col('finalScore') > 50, True).otherwise(False))#mai adaugam o coloana care va afisa true pentru cazul in care studentul a fost admis(finalScore > 50) sau false pentru cazul in care nu a fost admis(finalScore < 50)
df.show()#afisam rezultatele
#Salvam dataframe-ul modificat 
df.write.mode('overwrite').format('csv').option('header', 'true').save('/content/Students/StudentsPerformanceFormatted2')

+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|female|       group B| bachelor's degree|    standard|             none|       72|          72|          74|     72.67|   true|
|female|       group C|      some college|    standard|        completed|       69|          90|          88|     82.33|   true|
|female|       group B|   master's degree|    standard|             none|       90|          95|          93|     92.67|   true|
|  male|       group A|associate's degree|free/reduced|             none|       47|          57|          44|     49.33|  false|
|  male|       group C|      some college|    standard|             none|       76|          78| 

In [217]:
df.printSchema() #afisarea schemei datelor

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parentEducation: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- preparationCourse: string (nullable = true)
 |-- mathScore: integer (nullable = true)
 |-- readingScore: integer (nullable = true)
 |-- writingScore: integer (nullable = true)
 |-- finalScore: string (nullable = true)
 |-- admited: boolean (nullable = false)



In [218]:

 
#salvam locatia csv-ului formatat intr-o noua variabila
csv_path = "/content/Students/StudentsPerformanceFormatted2"

#incarcam noile date
df = spark.read.csv(csv_path, header=True, inferSchema=True) 

df.show()

+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|female|       group B| bachelor's degree|    standard|             none|       72|          72|          74|     72.67|   true|
|female|       group C|      some college|    standard|        completed|       69|          90|          88|     82.33|   true|
|female|       group B|   master's degree|    standard|             none|       90|          95|          93|     92.67|   true|
|  male|       group A|associate's degree|free/reduced|             none|       47|          57|          44|     49.33|  false|
|  male|       group C|      some college|    standard|             none|       76|          78| 